In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

from tools_cv import *
from tools_kinematics import *

In [2]:
# Constants
A = get_sym('A')  # car length
B = get_sym('B')  # car width / track
C = get_sym('C')  # car height
l = get_sym('l')  # wheelbase
r = get_sym('r')  # wheel radius
R = get_sym('R')  # turning radius (Ackerman)

X = get_sym(r'X')
X_dot = get_sym(r'X', dot=1)

Y = get_sym(r'Y')
Y_dot = get_sym(r'Y', dot=1)

Z = get_sym(r'Z')
Z_dot = get_sym(r'Z', dot=1)

theta = get_sym(r'\theta')
theta_dot = get_sym(r'\theta', dot=1)

alias = {}

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({
    sp.sin(theta): sp.symbols(r"s\theta"),
    sp.cos(theta): sp.symbols(r"c\theta"),
})

In [3]:
wheels = ['FL', 'FR', 'RL', 'RR']
L_FL = L_FR = B / 2
L_RL = L_RR = sp.sqrt(B**2 / 4 + l**2)
all_L = [L_FL, L_FR, L_RL, L_RR]

for wheel in wheels:
    alpha = get_sym(r'\alpha', wheel)
    exec(f"alpha_{wheel} = alpha")
    
    beta = get_sym(r'\beta', wheel)
    exec(f"beta_{wheel} = beta")
    
    phi = get_sym(r'\phi', wheel, dot=1)
    exec(f"phi_{wheel} = phi")
    
    # display cos(beta) as c\theta, sin(beta) as s\theta
    alias.update({
        sp.sin(beta): sp.symbols(r"s\beta_{%s}" % wheel),
        sp.cos(beta): sp.symbols(r"c\beta_{%s}" % wheel),
    })

In [4]:
# Overwrite constants
alpha_FL = sp.pi/2
alpha_FR = -sp.pi/2
alpha_RL = sp.pi/2 + sp.atan(2*l/B)
alpha_RR = -sp.pi/2 - sp.atan(2*l/B)

beta_RL = -sp.atan(2*l/B)
beta_RR = sp.atan(2*l/B) + sp.pi

In [5]:
RI0 = rotation(theta, 'z')
R0I = RI0.T
R0I_sym = get_sym('R', 'I', '0')
RI0_sym = get_sym('R', '0', 'I')
eqnprint(R0I_sym, alias, R0I)

P_dot = sp.Matrix([X_dot, Y_dot, theta_dot])
P_sym = get_sym('P', '', 'I', dot=1)
eqnprint(P_sym, alias, P_dot)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
# rolling condition
M_rolling_FL = get_mat_rolling(alpha_FL, beta_FL, L_FL)
M_rolling_FR = get_mat_rolling(alpha_FR, beta_FR, L_FR)
M_rolling_RL = get_mat_rolling(alpha_RL, beta_RL, L_RL)
M_rolling_RR = get_mat_rolling(alpha_RR, beta_RR, L_RR)

M_rolling = sp.Matrix([M_rolling_FL, M_rolling_FR,
                       M_rolling_RL, M_rolling_RR])

# so sliding condition
M_no_sliding_FL = get_mat_no_sliding(alpha_FL, beta_FL, L_FL)
M_no_sliding_FR = get_mat_no_sliding(alpha_FR, beta_FR, L_FR)
M_no_sliding_RL = get_mat_no_sliding(alpha_RL, beta_RL, L_RL)
M_no_sliding_RR = get_mat_no_sliding(alpha_RR, beta_RR, L_RR)

M_no_sliding = sp.Matrix([M_no_sliding_FL, M_no_sliding_FR,
                          M_no_sliding_RL, M_no_sliding_RR])

print('Rolling condition:')
left = [M_rolling,
        R0I_sym,
        P_sym,
        get_sym('-'),
        get_sym('r'),
        get_sym('\phi', dot=1)]
eqnprint(left, alias, None , 0)

print('No-sliding condition:')
left = [M_no_sliding,
        R0I_sym,
        P_sym,
        ]
eqnprint(left, alias, None , 0)

Rolling condition:


<IPython.core.display.Math object>

No-sliding condition:


<IPython.core.display.Math object>

In [8]:
M_angle = sp.Matrix([M_rolling_RL, M_rolling_RR, M_no_sliding_RL])
M_r = sp.eye(2)*r
M_r = M_r.row_insert(len(M_r), sp.zeros(1, 2))
M_phi = sp.Matrix([phi_RL, phi_RR])


left = [M_angle,
        R0I_sym,
        P_sym,
        get_sym('-'),
        M_r,
        M_phi]
eqnprint(left, alias, None , 0)

<IPython.core.display.Math object>

## Forward Kinematics

In [9]:
M_temp = M_angle.inv() * M_r
eqnprint(P_sym, alias, [RI0_sym,
                        M_temp,
                        M_phi])

M_temp = M_temp * M_phi
eqnprint(P_sym, alias, [RI0_sym,
                        M_temp])

P_I_dot = RI0 * M_temp
eqnprint(P_sym, alias, P_I_dot)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Inverse Kinematics

In [10]:
M_temp_left = M_r.T * M_r
M_temp_right = M_r.T * M_angle

left = [M_temp_left, M_phi]
right = [M_temp_right, R0I_sym, P_sym]
eqnprint(left, alias, right)

M_temp = M_temp_left.inv() * M_temp_right
right = [M_temp, R0I_sym, P_sym]
eqnprint(M_phi, alias, right)

M_inverse = M_temp * R0I
right = [M_inverse, P_sym]
eqnprint(M_phi, alias, right)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

From above, by knowing $\dot{^IP}$, we can know the values of $\dot{\phi_{RL}}$ and $\dot{\phi_{RR}}$.

We can then use them to solve $R$, $\beta_{FL}$, $\beta_{FL}$, $\dot{\phi_{FL}}$ and $\dot{\phi_{FR}}$.

In [11]:
# Turning radius
R_inv = B/2 * ((phi_RL + phi_RR) / (phi_RL - phi_RR))
eqnprint(R, alias, R_inv)

# front wheels beta
beta_FL_inv = -sp.atan(l / (R + B/2))
eqnprint(beta_FL, alias, beta_FL_inv)
beta_FR_inv = -sp.atan(l / (R - B/2))
eqnprint(beta_FR, alias, beta_FR_inv)

# front wheels speed
phi_FL_inv = theta_dot * sp.sqrt(l**2 + (R+B/2)**2) / r
eqnprint(phi_FL, alias, phi_FL_inv)
phi_FR_inv = theta_dot * sp.sqrt(l**2 + (R-B/2)**2) / r
eqnprint(phi_FR, alias, phi_FR_inv)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Validation

Starting by forward kinematics, making some random values of $\dot{\phi_{RL}}$ and $\dot{\phi_{RR}}$.

In [12]:
# Constants
A_vel = 3.6  # car length
B_vel = 2.6
C_val = 1.5
l_val = 2
r_val = 0.6

phi_RL_val = 0.01
phi_RR_val = 0.02

theta_val = np.pi/4

In [13]:
values = {A: A_vel, B: B_vel, C: C_val, l: l_val, r: r_val,
          phi_RL: phi_RL_val, phi_RR: phi_RR_val, theta: theta_val}

In [14]:
P_I_dot_vals = P_I_dot.subs(values)
eqnprint(get_sym('P', 'I', '', 1), alias, P_I_dot_vals)

<IPython.core.display.Math object>

In [15]:
rear_phi_values = M_inverse.multiply(P_I_dot_vals).subs(values)
phi_RL_val = rear_phi_values[0]
phi_RR_val = rear_phi_values[1]

eqnprint(phi_RL, alias, phi_RL_val)
eqnprint(phi_RR, alias, phi_RR_val)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Same as the initial values